In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm, global_add_pool, ChebConv, global_max_pool, SAGPooling, GATConv, GATv2Conv, TransformerConv, SuperGATConv, global_mean_pool, Linear
from torch.nn import BatchNorm1d
from math import floor
import torch
import torch_geometric
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

In [22]:
class EEGConvNetMini(nn.Module):
    """Same as EEGGraphConvNet but with fewer 
    convolutional layers
    """
    def __init__(self, reduced_sensors=True, sfreq=None, batch_size=256, input_size=6):
        super(EEGConvNetMini, self).__init__()
        # Define and initialize hyperparameters
        self.sfreq = sfreq
        self.batch_size = batch_size
        self.input_size = input_size
        
        # Layers definition
        # Graph convolutional layers
        self.conv1 = GCNConv(self.input_size, 16, cached=True, normalize=False)
        
        # Batch normalization
        self.batch_norm = BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        # Fully connected layers
        self.fc1 = nn.Linear(16, 8)
        self.fc2 = nn.Linear(8, 4)
        self.fc3 = nn.Linear(4, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        # Perform all graph convolutions
        x = F.leaky_relu(self.conv1(x, edge_index, edge_weigth), negative_slope=0.01)
        #x = F.dropout(x, p=0.2, training=self.training)
        
        # Perform batch normalization
        x = F.leaky_relu(self.batch_norm(x), negative_slope=0.01)
        #x = F.dropout(batch_norm_out, p=0.2, training=self.training)
        
        # Global add pooling
        mean_pool = global_add_pool(x, batch=batch)
        
        # Apply fully connected layters
        out = F.leaky_relu(self.fc1(mean_pool), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        
        out = F.leaky_relu(self.fc3(out))
        return out
    
    

In [42]:
def create_corrected_data_list(path):
    data_list = list()
    for file in path.iterdir():
        data_list.append(torch.load(file))
    corrected_data_list = list()
    for data in data_list:
    # print(data)
        data = torch_geometric.data.Data(
            x=torch.tensor(data.x),
            edge_index=torch.tensor(data.edge_index),
            edge_attr=torch.tensor(data.edge_attr),
            label=torch.tensor(data.label),
        )
        corrected_data_list.append(data)
       
       
    rm = [
      7,
      14+1,
      14+2,
      17+3,
      17+4,
      26+5,
      38+6,
      54+7,
      65+8,
      69+9
      ]

    dl = list()
    start = 0
    for r in rm:
        dl.extend(corrected_data_list[start:r])
        start = r + 1

    dl.extend(corrected_data_list[start:])
    dl_filterd = list()
    for data in dl:
        if data.label == 2:
            # print(data.label)
            # if data.label == 2:
            data.label = torch.tensor(1)
        dl_filterd.append(data)

    len(dl_filterd)
    return dl_filterd

def print_classification_report(y_pred, y_true):
    print(classification_report(y_true, y_pred))
            
    

## Moments Pearson

In [24]:
path = Path('graphs/moments_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_31200\765743231.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [28]:
model = EEGConvNetMini(input_size=6)

In [29]:
data = dl_filterd[0]

In [32]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 326


EEGConvNetMini(
  (conv1): GCNConv(6, 16)
  (batch_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=16, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=2, bias=True)
)

In [33]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.695010	 acc=0.653846
Test  : epoch=1	 loss=0.968978	 acc=0.562500

Train : epoch=2	 loss=0.937180	 acc=0.653846
Test  : epoch=2	 loss=2.036176	 acc=0.562500

Train : epoch=3	 loss=1.343597	 acc=0.653846
Test  : epoch=3	 loss=7.810587	 acc=0.500000

Train : epoch=4	 loss=2.027792	 acc=0.653846
Test  : epoch=4	 loss=13.934993	 acc=0.500000

Train : epoch=5	 loss=2.729467	 acc=0.653846
Test  : epoch=5	 loss=19.076998	 acc=0.500000

Train : epoch=6	 loss=3.034702	 acc=0.653846
Test  : epoch=6	 loss=19.996005	 acc=0.500000

Train : epoch=7	 loss=3.058595	 acc=0.653846
Test  : epoch=7	 loss=19.979201	 acc=0.500000

Train : epoch=8	 loss=3.055644	 acc=0.653846
Test  : epoch=8	 loss=19.921308	 acc=0.500000

Train : epoch=9	 loss=3.054728	 acc=0.653846
Test  : epoch=9	 loss=19.334044	 acc=0.500000

Train : epoch=10	 loss=3.066182	 acc=0.653846
Test  : epoch=10	 loss=18.738665	 acc=0.500000



In [34]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Moments PLI

In [36]:
path = Path('graphs/moments_pli/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_31200\765743231.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [37]:
model = EEGConvNetMini(input_size=6)

In [38]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 326


EEGConvNetMini(
  (conv1): GCNConv(6, 16)
  (batch_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=16, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=2, bias=True)
)

In [39]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.876429	 acc=0.641026
Test  : epoch=1	 loss=36.490707	 acc=0.625000

Train : epoch=2	 loss=1.310649	 acc=0.653846
Test  : epoch=2	 loss=28.235909	 acc=0.500000

Train : epoch=3	 loss=1.849130	 acc=0.653846
Test  : epoch=3	 loss=22.191071	 acc=0.562500

Train : epoch=4	 loss=2.665465	 acc=0.653846
Test  : epoch=4	 loss=64.719284	 acc=0.500000

Train : epoch=5	 loss=3.145089	 acc=0.653846
Test  : epoch=5	 loss=93.754211	 acc=0.500000

Train : epoch=6	 loss=3.406879	 acc=0.653846
Test  : epoch=6	 loss=136.812743	 acc=0.500000

Train : epoch=7	 loss=3.412288	 acc=0.653846
Test  : epoch=7	 loss=138.711183	 acc=0.500000

Train : epoch=8	 loss=3.517989	 acc=0.653846
Test  : epoch=8	 loss=93.883008	 acc=0.500000

Train : epoch=9	 loss=3.300659	 acc=0.653846
Test  : epoch=9	 loss=96.688724	 acc=0.500000

Train : epoch=10	 loss=3.641460	 acc=0.653846
Test  : epoch=10	 loss=91.969898	 acc=0.500000



In [40]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Raw Pearson

In [43]:
path = Path('graphs/raw_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_31200\114600516.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_31200\114600516.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [44]:
model = EEGConvNetMini(input_size=153550)

In [45]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 2457030


EEGConvNetMini(
  (conv1): GCNConv(153550, 16)
  (batch_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=16, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=2, bias=True)
)

In [46]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=2.753490	 acc=0.551282
Test  : epoch=1	 loss=95.415007	 acc=0.500000

Train : epoch=2	 loss=3.404291	 acc=0.653846
Test  : epoch=2	 loss=55.725266	 acc=0.437500

Train : epoch=3	 loss=2.866058	 acc=0.653846
Test  : epoch=3	 loss=78.643512	 acc=0.437500

Train : epoch=4	 loss=3.428240	 acc=0.653846
Test  : epoch=4	 loss=59.969589	 acc=0.437500

Train : epoch=5	 loss=3.118471	 acc=0.653846
Test  : epoch=5	 loss=60.295269	 acc=0.375000

Train : epoch=6	 loss=2.944756	 acc=0.653846
Test  : epoch=6	 loss=29.285337	 acc=0.312500

Train : epoch=7	 loss=2.881629	 acc=0.653846
Test  : epoch=7	 loss=15.170636	 acc=0.500000

Train : epoch=8	 loss=2.804063	 acc=0.653846
Test  : epoch=8	 loss=11.137017	 acc=0.500000

Train : epoch=9	 loss=2.685399	 acc=0.653846
Test  : epoch=9	 loss=36.964104	 acc=0.312500

Train : epoch=10	 loss=2.596020	 acc=0.653846
Test  : epoch=10	 loss=68.638241	 acc=0.312500



In [47]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.38      0.62      0.48         8

    accuracy                           0.31        16
   macro avg       0.19      0.31      0.24        16
weighted avg       0.19      0.31      0.24        16



## Raw PLI

In [48]:
path = Path('graphs/raw_pli/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_31200\114600516.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_31200\114600516.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [49]:
model = EEGConvNetMini(input_size=153550, sfreq=500)

In [50]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 2457030


EEGConvNetMini(
  (conv1): GCNConv(153550, 16)
  (batch_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=16, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=2, bias=True)
)

In [51]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=1.585948	 acc=0.641026
Test  : epoch=1	 loss=47.135586	 acc=0.562500

Train : epoch=2	 loss=2.149640	 acc=0.653846
Test  : epoch=2	 loss=53.580137	 acc=0.625000

Train : epoch=3	 loss=2.695424	 acc=0.653846
Test  : epoch=3	 loss=59.521744	 acc=0.437500

Train : epoch=4	 loss=2.676652	 acc=0.653846
Test  : epoch=4	 loss=58.465616	 acc=0.500000

Train : epoch=5	 loss=2.660961	 acc=0.653846
Test  : epoch=5	 loss=50.986474	 acc=0.500000

Train : epoch=6	 loss=2.677856	 acc=0.653846
Test  : epoch=6	 loss=49.414424	 acc=0.500000

Train : epoch=7	 loss=2.606995	 acc=0.653846
Test  : epoch=7	 loss=49.729958	 acc=0.500000

Train : epoch=8	 loss=2.582192	 acc=0.653846
Test  : epoch=8	 loss=49.759071	 acc=0.562500

Train : epoch=9	 loss=2.677484	 acc=0.653846
Test  : epoch=9	 loss=49.931299	 acc=0.562500

Train : epoch=10	 loss=2.562927	 acc=0.653846
Test  : epoch=10	 loss=41.128629	 acc=0.562500



In [52]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       1.00      0.12      0.22         8
           1       0.53      1.00      0.70         8

    accuracy                           0.56        16
   macro avg       0.77      0.56      0.46        16
weighted avg       0.77      0.56      0.46        16



## PSD CSD

In [53]:
path = Path('graphs/psd_csd/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_31200\114600516.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [54]:
model = EEGConvNetMini(sfreq=500, input_size=6)

In [56]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 326


EEGConvNetMini(
  (conv1): GCNConv(6, 16)
  (batch_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=16, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=2, bias=True)
)

In [58]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.822290	 acc=0.653846
Test  : epoch=1	 loss=0.726788	 acc=0.500000

Train : epoch=2	 loss=1.021170	 acc=0.653846
Test  : epoch=2	 loss=0.765818	 acc=0.500000

Train : epoch=3	 loss=1.420816	 acc=0.653846
Test  : epoch=3	 loss=1.370760	 acc=0.500000

Train : epoch=4	 loss=3.003399	 acc=0.653846
Test  : epoch=4	 loss=3.674813	 acc=0.500000

Train : epoch=5	 loss=3.564597	 acc=0.653846
Test  : epoch=5	 loss=4.208704	 acc=0.500000

Train : epoch=6	 loss=3.563034	 acc=0.653846
Test  : epoch=6	 loss=4.221987	 acc=0.500000

Train : epoch=7	 loss=3.556274	 acc=0.653846
Test  : epoch=7	 loss=4.238070	 acc=0.500000

Train : epoch=8	 loss=3.550967	 acc=0.653846
Test  : epoch=8	 loss=4.249033	 acc=0.500000

Train : epoch=9	 loss=3.545699	 acc=0.653846
Test  : epoch=9	 loss=4.256005	 acc=0.500000

Train : epoch=10	 loss=3.541015	 acc=0.653846
Test  : epoch=10	 loss=4.260894	 acc=0.500000



In [59]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## PSD Pearson

In [60]:
path = Path('graphs/psd_pearson/')
dl_filterd = create_corrected_data_list(path)
train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=1, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=10, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_31200\114600516.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [61]:
model = EEGConvNetMini(sfreq=500, input_size=6)

In [62]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()

print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 326


EEGConvNetMini(
  (conv1): GCNConv(6, 16)
  (batch_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=16, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=4, bias=True)
  (fc3): Linear(in_features=4, out_features=2, bias=True)
)

In [63]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=1.663577	 acc=0.282051
Test  : epoch=1	 loss=3.404387	 acc=0.625000

Train : epoch=2	 loss=0.695628	 acc=0.615385
Test  : epoch=2	 loss=1.955927	 acc=0.500000

Train : epoch=3	 loss=0.688634	 acc=0.653846
Test  : epoch=3	 loss=1.480324	 acc=0.562500

Train : epoch=4	 loss=0.786435	 acc=0.653846
Test  : epoch=4	 loss=0.809998	 acc=0.562500

Train : epoch=5	 loss=1.030084	 acc=0.653846
Test  : epoch=5	 loss=1.641907	 acc=0.562500

Train : epoch=6	 loss=1.709157	 acc=0.653846
Test  : epoch=6	 loss=5.226518	 acc=0.562500

Train : epoch=7	 loss=2.932782	 acc=0.653846
Test  : epoch=7	 loss=9.041891	 acc=0.562500

Train : epoch=8	 loss=3.366873	 acc=0.653846
Test  : epoch=8	 loss=9.541908	 acc=0.562500

Train : epoch=9	 loss=3.342110	 acc=0.653846
Test  : epoch=9	 loss=8.950352	 acc=0.562500

Train : epoch=10	 loss=3.309732	 acc=0.653846
Test  : epoch=10	 loss=9.222539	 acc=0.562500



In [64]:
print_classification_report(y_pred, y_true)

              precision    recall  f1-score   support

           0       1.00      0.12      0.22         8
           1       0.53      1.00      0.70         8

    accuracy                           0.56        16
   macro avg       0.77      0.56      0.46        16
weighted avg       0.77      0.56      0.46        16

